# Basic classification: Classify images of clothing
A simple MNIST classification service example using MLFlow(no benchmarks)


![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=tensorflow&ea=tensorflow_2_fashion_mnist&dt=tensorflow_2_fashion_mnist)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# add venv PATH to shell command PATH
import sys, os
if sys.base_prefix not in os.environ['PATH']:
    os.environ['PATH'] = f"{sys.base_prefix}/bin:{os.environ['PATH']}"

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import io

# TensorFlow
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
print(tf.__version__)

2.1.0


In [3]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(_train_images, train_labels), (_test_images, test_labels) = fashion_mnist.load_data()
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
train_images = _train_images / 255.0
test_images = _test_images / 255.0

In [6]:
class FashionMnist(tf.keras.Model):
    def __init__(self):
        super(FashionMnist, self).__init__()
        self.cnn = tf.keras.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28, )),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(10, activation='softmax')
        ])
    
    @tf.function(input_signature=[tf.TensorSpec(shape=(None, 28, 28), dtype=np.float32)])
    def call(self, inputs):
        return self.cnn(inputs)


In [7]:
model = FashionMnist()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=1)

Train on 60000 samples
60000/60000 [==============================] - 6s 103us/sample - loss: 0.4984 - accuracy: 0.8256


In [8]:
tmpdir = 'mlflow_tmp_2'
tf_model_path = os.path.join(str(tmpdir), "tf.pkl")
tf.saved_model.save(model, tf_model_path)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: mlflow_tmp/tf.pkl/assets


In [40]:
%%writefile benchmark_mlflow_pyfunc_2.py
from __future__ import print_function

import os
import pickle

import pandas as pd
import numpy as np
import pytest
import six

import tensorflow as tf

import mlflow
import mlflow.pyfunc
import mlflow.pyfunc.model
from mlflow.models import Model


def _load_pyfunc(path):
    tf_model = tf.saved_model.load(path)
    class Model:
        class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
                       'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
        def predict(self, inputs):
            inputs = tf.constant(inputs.to_numpy().reshape([-1, 28, 28]), dtype=tf.float32)
            outputs = tf_model(inputs)
            outputs = tf.argmax(outputs, axis=1)
            return [self.class_names[i] for i in outputs.numpy()]
    return Model()
        


if __name__ == '__main__':
    tmpdir = 'mlflow_tmp'
    tf_model_path = os.path.join(str(tmpdir), "tf.pkl")
    model_path = os.path.join(str(tmpdir), "model")

    model_config = Model(run_id="test")
    mlflow.pyfunc.save_model(path=model_path,
                             data_path=tf_model_path,
                             loader_module=os.path.basename(__file__)[:-3],
                             code_path=[__file__],
                             mlflow_model=model_config)

    reloaded_model = mlflow.pyfunc.load_pyfunc(model_path)
    print(reloaded_model)

    fashion_mnist = tf.keras.datasets.fashion_mnist
    (_train_images, train_labels), (_test_images, test_labels) = fashion_mnist.load_data()
    train_images = _train_images / 255.0
    test_images = _test_images / 255.0
    inputs = pd.DataFrame(np.reshape(test_images[:3], [-1, 28 * 28]))
    r = reloaded_model.predict(inputs)
    print(r)

Overwriting benchmark_mlflow_pyfunc.py


In [41]:
!rm -r {tmpdir}/model
!python benchmark_mlflow_pyfunc_2.py

benchmark_mlflow_pyfunc.py:45: DeprecationWarning: .. Warning:: ``mlflow.pyfunc.load_pyfunc`` is deprecated since 1.0. This method will be removed in a near future release. Use ``mlflow.pyfunc.load_model`` instead.
  reloaded_model = mlflow.pyfunc.load_pyfunc(model_path)
2020-03-12 16:43:48.890588: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2020-03-12 16:43:48.916035: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2712000000 Hz
2020-03-12 16:43:48.916831: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55f1ad531970 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-03-12 16:43:48.916874: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-03-12 16:43:48.917040: I tensorflow/core/common_runtime/process_util.cc:147] Creating new thre

In [34]:
!mlflow models serve -m {tmpdir}/model

/opt/anaconda3/envs/bentoml-dev-py36/lib/python3.6/site-packages/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
2020/03/12 15:55:57 INFO mlflow.models.cli: Selected backend for flavor 'python_function'
2020/03/12 15:55:57 INFO mlflow.pyfunc.backend: === Running command 'gunicorn --timeout=60 -b 127.0.0.1:5000 -w 1 ${GUNICORN_CMD_ARGS} -- mlflow.pyfunc.scoring_server.wsgi:app'
[2020-03-12 15:55:57 +0800] [26760] [INFO] Starting gunicorn 20.0.4
[2020-03-12 15:55:57 +0800] [26760] [INFO] Listening at: http://127.0.0.1:5000 (26760)
[2020-03-12 15:55:57 +0800] [26760] [INFO] Using worker: sync
[2020-03-12 15:55:57 +0800] [26763] [INFO] Booting worker with pid: 26763
/opt/anaconda3/envs/bentoml-dev-py36/lib/python3.6/site-packages/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alte

# Test with requests

In [44]:
import base64
import json
import requests
import pandas as pd

with open("test.png", "rb") as f:
    img_bytes = f.read()
img_b64 = base64.b64encode(img_bytes).decode()


headers = {"content-type": "application/json"}
raw_data = np.reshape(test_images[:10], (-1, 28 * 28))
print(raw_data.shape)
data = pd.DataFrame(raw_data, columns=map(str, range(raw_data.shape[1]))).to_json(orient='split')

json_response = requests.post(f'http://127.0.0.1:5000/invocations', data=data, headers=headers)
print(json_response)
print(json_response.json())

(10, 784)
<Response [200]>
['Ankle boot', 'Pullover', 'Trouser', 'Trouser', 'Shirt', 'Trouser', 'Coat', 'Shirt', 'Sandal', 'Sneaker']
